In [1]:
import sys
sys.path.append("../")

import torch
import torchvision
from torch.utils.data import DataLoader

from scl.encoders import resnet50
from scl.aug import get_inference_transforms

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/wavelet/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
IMAGE_SIZE = 96
transform = get_inference_transforms(image_size=(IMAGE_SIZE, IMAGE_SIZE))

train_ds = torchvision.datasets.STL10("../data/",
                                  split='train',
                                  transform=transform,
                                  download=True)
val_ds = torchvision.datasets.STL10("../data",
                                  split='test',
                                  transform=transform,
                                  download=True)

train_loader = DataLoader(train_ds,
                          batch_size=128,
                          num_workers=4)
val_loader = DataLoader(val_ds,
                       batch_size=128,
                       num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
len(train_ds), len(val_ds)

(5000, 8000)

In [5]:
ckpt = torch.load("../models/encoder_sigcon_alpha0.1_gamma0.0_300epochs.pth")

# model = resnet18()
model = resnet50()
model.load_state_dict(ckpt)

model = model.eval().to(device)

In [6]:
from tqdm.auto import tqdm
import numpy as np
import torch.nn.functional as F

def get_embs_labels(dl):
    idx = 0
    embs, labels = [], []
    for idx, (images, targets) in enumerate(tqdm(dl)):
        with torch.no_grad():
            images = images.to(device)
            out = model(images)
            features = out.cpu().detach()
            features = F.normalize(features, p=2, dim=-1)
            embs.extend(features.tolist())
            labels.extend(targets.cpu().detach().tolist())
    return np.array(embs), np.array(labels)

In [7]:
embeddings, labels = get_embs_labels(train_loader)
embeddings_val, labels_val = get_embs_labels(val_loader)

100%|██████████| 63/63 [00:01<00:00, 38.28it/s]


In [8]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def eval():
    X_train, X_test = embeddings, embeddings_val
    y_train, y_test = labels, labels_val

    print("train", X_train.shape[0], len(y_train))
    print("test", X_test.shape[0], len(y_test))
    
    clf = LogisticRegression(max_iter=100)
    clf = CalibratedClassifierCV(clf)
    
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    
    print("Accuracy: ", acc)
    print("Confusion matrix: \n", conf_matrix)
    print("Classification report: \n", class_report)
    
    y_pred_train = clf.predict(X_train)
    class_report = classification_report(y_train, y_pred_train)
    print("Classification report train: \n", class_report)

In [23]:
# encoder_sigcon_alpha0.1_gamma2.0_cosine_schedule20k.pth
eval()

train 5000 5000
test 8000 8000
Accuracy:  0.856
Confusion matrix: 
 [[757  11   5   1   0   1   1   0  13  11]
 [ 11 666   0  49   8  21   7  38   0   0]
 [  2   4 756   0   0   0   0   0   1  37]
 [  0  42   0 547  45 109   4  52   1   0]
 [  0  20   1  38 669  25  33  14   0   0]
 [  0  15   0  41  44 576  57  65   1   1]
 [  1   3   0   7  30  38 697  19   0   5]
 [  0  36   0  57  19  40   4 644   0   0]
 [  7   0   0   2   0   0   0   0 782   9]
 [  8   2  19   2   0   0   0   1  14 754]]
Classification report: 
               precision    recall  f1-score   support

           0       0.96      0.95      0.95       800
           1       0.83      0.83      0.83       800
           2       0.97      0.94      0.96       800
           3       0.74      0.68      0.71       800
           4       0.82      0.84      0.83       800
           5       0.71      0.72      0.72       800
           6       0.87      0.87      0.87       800
           7       0.77      0.81      0.79